In [1]:
import pandas as pd
import numpy as np

Episode 1 : **LE CLEANING**

Import de l'ensemble des datas, puis netoyage de celles-ci. On applique certains filtres, supression de colonnes non nécéssaires et des colonnes déja filtrées.

In [ ]:
#import IMDB_title_ratings
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', compression='gzip')

In [ ]:
#import IMDB_name_basics
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', compression='gzip')

In [ ]:
#import IMDB_title_basics
url_imdb_title_basics = "https://datasets.imdbws.com/title.basics.tsv.gz"
df_imdb_title_basics = pd.read_csv(url_imdb_title_basics, sep='\t', compression='gzip')

#Le cleaning
df_imdb_title_basics = df_imdb_title_basics[(df_imdb_title_basics['titleType'] == 'movie') & 
                                            (df_imdb_title_basics['vote_average'] > 5)]

df_imdb_title_basics = df_imdb_title_basics[df_imdb_title_basics['isAdult'] == 0]
df_imdb_title_basics = df_imdb_title_basics[df_imdb_title_basics['startYear'] > 1950]
df_imdb_title_basics.drop(['isAdult', 'endYear', 'primaryTitle'], axis=1, inplace=True)
df_imdb_title_basics


In [ ]:
#import IMDB_title_akas
url_imdb_title_akas = "https://datasets.imdbws.com/title.akas.tsv.gz"
df_imdb_title_akas = pd.read_csv(url_imdb_title_akas, sep='\t', compression='gzip')

#Le cleaning
df_imdb_title_akas = df_imdb_title_akas[df_imdb_title_akas['region'] == 'FR']
df_imdb_title_akas.drop(['ordering', 'language', 'types', 'attributes', 'isOriginalTitle', 'region'], axis=1, inplace=True)
df_imdb_title_akas

In [ ]:
#import IMDB_title_principals
url_imdb_title_principals = "https://datasets.imdbws.com/title.principals.tsv.gz"
df_imdb_title_principals = pd.read_csv(url_imdb_title_principals, sep='\t', compression='gzip')

#Le cleaning
df_imdb_title_principals = df_imdb_title_principals[df_imdb_title_principals['ordering'] < 10]
df_imdb_title_principals.drop(['job', 'ordering'], axis=1, inplace=True)


Episode 2 : **LE MERGING**

Ici on fusionne les tables une à une. D'abord sur ce qui concerne les infos sur les films, puis qui concerne les "crew". Enfin, on rassemble toutes les infos des "crew" dans un dictionnaire pour l'ajouter dans une colonne de la table film. Ainsi chaque film dispose d'une colonne avec l'ensemble des infos sur le crew du film.

In [ ]:
# merge de title_basisc et akas
df_bascisakas = pd.merge(
    df_imdb_title_basics,
    df_imdb_title_akas,
    left_on='Tconst',
    right_on='titleId',
    how='inner' 
)

In [ ]:
#Merge df_basicsakas avec ratings
df_basicsakasratings = pd.merge(
    df_bascisakas,
    df_title_ratings,
    on='Tconst', 
    how='inner'  
)

In [ ]:
#Merge name_basics avec titles_principals
df_basicsprincipals = pd.merge(
    df_imdb_title_principals,
    df_name_basics
    on='nconst',  
    how='inner'  
)

Episode 3 : **LA CONSOLIDATION**

In [ ]:
#création d'un dictionnaire avec les infos de df_basicsprincipals
grouped_crew = df_basicsprincipals.groupby('titleId').apply(
    lambda x: x.to_dict(orient='records')
).to_dict()

In [ ]:
#Ajout d'une nouvelle colonne dans df_basicsakasratings
df_basicsakasratings['crew_details'] = df_basicsakasratings['Tconst'].map(grouped_crew)

In [ ]:
#On renomme en df_final
df_final = df_basicsakasratings